In [47]:
import torch
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
import numpy as np

In [48]:
%run ConV_VAE.ipynb
%run loader.ipynb

In [49]:
device = 'cuda'

In [50]:
def test(epoch, model, loader):
    model.eval()
    test_losses = []
    with torch.no_grad():
        for i, data in enumerate(loader):
            #data = data.transpose(0,1)
            inputs, labels = data
            inputs = inputs.to(device)
            inputs = inputs.unsqueeze(dim=0).transpose(0,1)
            recon_batch = model(inputs)
            test_loss = ((inputs - recon_batch) ** 2).sum() + model.encoder.kl
            test_losses.append(test_loss.item())
            if i == 0:
                n = min(inputs.size(0), 8)
                comparison = torch.cat([inputs[:n],
                                        recon_batch.view(128, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                           '../results/reconstruction/' + str(epoch) + '.png', nrow=n)
    #print('====> Test set loss: {:.4f}'.format(test_loss))
    return np.mean(test_losses)

In [51]:
# normalize = transforms.Normalize(160, 50)
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     normalize
# ])
# mnist_data = CustomImageDataset('../Data/sign_mnist_test.csv', transform=transform)
# # mnist_data = datasets.MNIST('./data',
# #                             transform=transforms.ToTensor(),
# #                             download=True)

Put it into a dataloader for easier handling in pytorch

In [52]:
# mnist_loader = torch.utils.data.DataLoader(mnist_data, batch_size=128, shuffle=False)

In [53]:
# mnist_loader = torch.utils.data.DataLoader(mnist_data, batch_size=128, shuffle=False)

In [54]:
# model = ConvVarAutoencoder().to(device)
# model.load_state_dict(torch.load("model_good_50.pt", map_location=device))
# writer = SummaryWriter()
# epochs = 5
# for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
#     # Test
#     test_loss = test(epoch, model, mnist_loader)
#     print(test_loss)
#     # Write metrics to Tensorboard
#     #writer.add_scalars("Loss", {'Train': test_loss}, epoch)